In [16]:
import networkx as nx
import random 
from IPython.display import display, HTML
import time
from ortools.linear_solver import pywraplp
import pandas as pd

In [17]:
def gerar_grafo_aleatorio(n, p, c):
    G = nx.gnp_random_graph(n, p)
    
    # Atribuir pesos aleatórios às arestas
    for u, v in G.edges():
        G[u][v]['weight'] = random.randint(1, 10)

    # Garante que os vértices centrais tenham vizinhos (não sejam isolados)
    centrais = {}
    candidatos = [v for v in G.nodes() if len(list(G.neighbors(v))) > 0]
    
    if len(candidatos) < c:
        raise ValueError("Não há vértices suficientes com vizinhos para serem centrais")

    # Seleciona c vértices centrais aleatórios e define graus mínimos aleatórios
    centrais_vertices = random.sample(candidatos, c)
    for i in centrais_vertices:
        centrais[i] = random.randint(2, c-1)
    
    return G, centrais


In [18]:
# Função para ler as instâncias do problema
def ler_instancia(caminho_arquivo):
    with open(caminho_arquivo, 'r') as f:
        # Ler a primeira linha (n, nc, m)
        primeira_linha = f.readline().strip().split()
        n = int(primeira_linha[0])
        nc = int(primeira_linha[1])
        m = int(primeira_linha[2])
        
        # Inicializar o grafo
        G = nx.Graph()
        
        # Ler os vértices centrais e seus graus mínimos
        centrais = {}
        for _ in range(nc):
            linha = f.readline().strip().split()
            vertice_central = int(linha[0])
            grau_minimo = int(linha[1])
            centrais[vertice_central] = grau_minimo
        

        # Ler as arestas e os custos
        for _ in range(m):
            linha = f.readline().strip().split()
            i = int(linha[0])
            j = int(linha[1])
            custo = int(linha[2])
            G.add_edge(i, j, weight=custo)
        
    return G, centrais

In [19]:
def combinacoes_sem_repeticao(it, r):
    # Converte o iterável em uma tupla para indexação
    pool = tuple(it)
    # Número total de elementos no pool
    n = len(pool)
    
    # Se o comprimento da combinação desejada é maior que o número de elementos,
    # não é possível gerar combinações
    if r > n:
        return
    
    # Inicializa os índices para a combinação inicial
    indices = list(range(r))
    
    # Gera a primeira combinação
    yield tuple(pool[i] for i in indices)
    
    while True:
        # Percorre os índices da combinação de trás para frente
        for i in reversed(range(r)):
            # Verifica se o índice atual pode ser incrementado
            if indices[i] != i + n - r:
                break
        else:
            # Se nenhum índice pode ser incrementado, todas as combinações foram geradas
            return
        
        # Incrementa o índice encontrado
        indices[i] += 1
        
        # Atualiza os índices subsequentes para garantir que estejam em ordem crescente
        for j in range(i + 1, r):
            indices[j] = indices[j - 1] + 1
        
        # Gera a próxima combinação
        yield tuple(pool[i] for i in indices)


def eliminacao_ciclos(G, d):
    start_time = time.time()

    # Criar solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Variáveis de decisão: x[u, v] = 1 se a aresta (u, v) é selecionada
    x = { (u, v): solver.BoolVar(f'x_{u}_{v}') for u, v in G.edges() }
    x.update({ (v, u): solver.BoolVar(f'x_{v}_{u}') for u, v in G.edges() })  # Para grafo não direcionado

    # Função objetivo: minimizar custo das arestas selecionadas
    solver.Minimize(solver.Sum(G.edges[u, v]['weight'] * x[u, v] for u, v in G.edges()))

    # Restrição 1: Árvore geradora no subgrafo dos centrais
    centrais = d.keys()
    solver.Add(solver.Sum(x[u, v] for u, v in G.edges() if u in centrais and v in centrais) == len(centrais) - 1)

    # Restrição 2: Eliminação de ciclos no subgrafo dos centrais (Subtour Elimination Constraints)
    for r in range(2, len(centrais)):
        for subset in combinacoes_sem_repeticao(centrais, r):
            solver.Add(solver.Sum(x[u, v] for u, v in combinacoes_sem_repeticao(subset, 2) if G.has_edge(u, v)) <= len(subset) - 1)

    # Restrição 3: Grau mínimo para os vértices centrais
    for v in centrais:
        solver.Add(solver.Sum(x[u, v] for u in G.neighbors(v)) >= d[v])

    # Restrição 4: Terminais conectados a apenas um vértice central
    terminals = set(G.nodes()) - set(centrais)
    for t in terminals:
        solver.Add(solver.Sum(x[t, v] for v in G.neighbors(t)) == 1)

    # Restrição 5: Garantir que as variáveis são binárias
    for u, v in G.edges():
        solver.Add(x[u, v] == x[v, u])

    # Limite de tempo para o solver
    solver.SetTimeLimit(1800000)  # 1800 segundos

    # Resolver o problema
    tempo_inicio = time.time()
    status = solver.Solve()
    tempo_resultado = time.time() - tempo_inicio
    
    if status == pywraplp.Solver.OPTIMAL:
        return tempo_resultado, "Solução ótima!"
    elif status == pywraplp.Solver.FEASIBLE:
         return tempo_resultado, "Solução viável!"
    else:
        return tempo_resultado, "Solução ótima não encontrada."


In [20]:
def rotulos_vertices(G, d):
    # Transformar o grafo em um grafo direcionado
    directed_G = nx.DiGraph()
    for u, v, data in G.edges(data=True):
        directed_G.add_edge(u, v, **data)  # Aresta u -> v
        directed_G.add_edge(v, u, **data)  # Aresta v -> u
    G = directed_G

    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Definir o nó raiz
    raiz = 0
    if raiz not in G.nodes():
        raiz = list(G.nodes())[0]

    # Variáveis de decisão: 1 se o arco (u, v) fizer parte da árvore, 0 caso contrário
    aresta_selecionada = {
        (u, v): solver.BoolVar(f'aresta_{u}_{v}')
        for u, v in G.edges()
    }

    # Função objetivo: minimizar o custo total das arestas selecionadas
    solver.Minimize(
        solver.Sum(G[u][v]['weight'] * aresta_selecionada[(u, v)] for u, v in G.edges())
    )

    # Variáveis de rótulos para os vértices
    rotulos = {
        v: solver.IntVar(0, 0, f'rotulo_{v}') if v == raiz else solver.IntVar(1, len(G.nodes()), f'rotulo_{v}')
        for v in G.nodes()
    }

    # Restrições para evitar ciclos
    num_vertices = len(G.nodes())
    for u, v in G.edges():
        solver.Add(rotulos[u] - rotulos[v] + num_vertices * aresta_selecionada[(u, v)] <= num_vertices - 1)

    # Restrições de grau mínimo para os vértices centrais
    for v in d:
        if v != raiz:
            solver.Add(solver.Sum(aresta_selecionada[(v, j)] for j in G.successors(v)) >= d[v] - 1)

    solver.Add(solver.Sum(aresta_selecionada[(raiz, j)] for j in G.successors(raiz)) >= d[raiz])

    # Restrições de grau de entrada: todos os vértices, exceto a raiz, devem ter 1 aresta de entrada
    for v in G.nodes():
        if v != raiz:
            solver.Add(solver.Sum(aresta_selecionada[(u, v)] for u in G.predecessors(v)) == 1)

    # Restrições para folhas: vértices que não são centrais devem ser folhas
    for v in G.nodes():
        if v not in d:
            solver.Add(solver.Sum(aresta_selecionada[(u, v)] for u in G.predecessors(v)) + solver.Sum(aresta_selecionada[(v, j)] for j in G.successors(v)) == 1)

    solver.SetTimeLimit(1800 * 1000)  # Limite de tempo de 1800 segundos

    inicio = time.time()
    status = solver.Solve()
    tempo_final = time.time() - inicio

    if status == pywraplp.Solver.OPTIMAL:
        # print("Solução ótima encontrada.")
        return tempo_final,"Solução ótima!"
    elif status == pywraplp.Solver.FEASIBLE:
        # print("Solução viável encontrada, mas pode não ser ótima.")
        return tempo_final,"Solução viável!"
    else:
        return tempo_final,"Solução não encontrada"

In [67]:
def comparar_formulacoes(caminho_arquivo):
    # Ler a instância
    G, centrais = ler_instancia(caminho_arquivo)
        
    # Eliminação de Ciclos
    tempo_eliminacao_ciclos,resultado1  = eliminacao_ciclos(G, centrais)
    
    # Rótulos nos Vértices
    tempo_rotulos_vertices,resultado2 = rotulos_vertices(G, centrais)

    return tempo_eliminacao_ciclos,resultado1, tempo_rotulos_vertices,resultado2


In [65]:
# Exemplo de uso com as instâncias
instancias = [
     'instancias/tb8ch4_0.txt',
     'instancias/tb8ch4_1.txt',
     'instancias/tb8ch8_0.txt',
     'instancias/tb8ch8_1.txt',
     'instancias/tb8ch10_0.txt',
     'instancias/tb8ch10_1.txt',
     'instancias/tb8ch15_0.txt',
     'instancias/tb8ch15_1.txt'
]
mensagem = "<h3>A partir das instâncias com 20 centrais ou mais, meu computador não consegue computar.</h3>"

# Lista para armazenar os dados de cada instância
resultados = []

for instancia in instancias:
    tempo_ciclos,resultado1, tempo_rotulos,resultado2 = comparar_formulacoes(instancia)
    resultados.append({
        'Instância': instancia,
        'Tempo Eliminação de Ciclos (s)': tempo_ciclos,
        'Resultado Eliminação de Ciclos': resultado1,
        'Tempo Rótulos nos Vértices (s)': tempo_rotulos,
        'Resultado Rótulos nos Vértices': resultado2
    })
# Criar DataFrame com os resultados
df = pd.DataFrame(resultados)
display(HTML(mensagem))
display(df)

{1: 3, 2: 3, 3: 3, 4: 3} 11
{1: 3, 2: 4, 3: 2, 4: 3} 11
{1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3} 21
{1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 4, 7: 2, 8: 2} 20
{1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3, 9: 3, 10: 3} 26
{1: 3, 2: 3, 3: 3, 4: 3, 5: 4, 6: 2, 7: 2, 8: 3, 9: 3, 10: 3} 25
{1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3, 9: 3, 10: 3, 11: 3, 12: 3, 13: 3, 14: 3, 15: 3} 38
{1: 3, 2: 2, 3: 3, 4: 3, 5: 3, 6: 3, 7: 2, 8: 4, 9: 3, 10: 3, 11: 3, 12: 2, 13: 4, 14: 3, 15: 3} 37


,Instância,Tempo Eliminação de Ciclos (s),Resultado Eliminação de Ciclos,Tempo Rótulos nos Vértices (s),Resultado Rótulos nos Vértices
0,instancias/tb8ch4_0.txt,0.001714,Solução ótima!,0.002902,Solução ótima!
1,instancias/tb8ch4_1.txt,0.002064,Solução ótima!,0.004933,Solução ótima!
2,instancias/tb8ch8_0.txt,0.010723,Solução ótima!,0.042949,Solução ótima!
3,instancias/tb8ch8_1.txt,0.009317,Solução ótima!,0.009952,Solução ótima!
4,instancias/tb8ch10_0.txt,0.042761,Solução ótima!,0.849910,Solução ótima!
5,instancias/tb8ch10_1.txt,0.042337,Solução ótima!,0.060773,Solução ótima!
6,instancias/tb8ch15_0.txt,2.371982,Solução ótima!,0.710174,Solução ótima!
7,instancias/tb8ch15_1.txt,2.259723,Solução ótima!,0.148872,Solução ótima!


In [68]:
try: 
    grafo1, centrais1 = gerar_grafo_aleatorio(10, 0.5, 3)  # número de vértices, probabilidade de aresta, número de centrais
    grafo2, centrais2 = gerar_grafo_aleatorio(10, 0.9, 3)

    # Executa as funções de eliminação de ciclos e rótulos nos vértices para os dois grafos
    tempo_ciclos1, resultado1 = eliminacao_ciclos(grafo1, centrais1)
    tempo_rotulos1, resultado2 = rotulos_vertices(grafo1, centrais1)

    tempo_ciclos2, resultado3 = eliminacao_ciclos(grafo2, centrais2)
    tempo_rotulos2, resultado4 = rotulos_vertices(grafo2, centrais2)

    resultados = []

    # Armazenando os resultados
    resultados.append({
        'Instância': 'grafo 1',
        'Tempo Eliminação de Ciclos (s)': tempo_ciclos1,
        'Resultado Eliminação de Ciclos': resultado1,
        'Tempo Rótulos nos Vértices (s)': tempo_rotulos1,
        'Resultado Rótulos nos Vértices': resultado2
    })

    resultados.append({
        'Instância': 'grafo 2',
        'Tempo Eliminação de Ciclos (s)': tempo_ciclos2,
        'Resultado Eliminação de Ciclos': resultado3,
        'Tempo Rótulos nos Vértices (s)': tempo_rotulos2,
        'Resultado Rótulos nos Vértices': resultado4
    })

    # Criar DataFrame com os resultados
    df = pd.DataFrame(resultados)

    # Exibir os resultados
    display(HTML("<h2>Pelo fato do grafo ser aléatoriom, as vezes ele quera a funções produzidas</h2>"))
    display(HTML("<h3>Comparação entre a Eliminação de Ciclos e Rótulos nos Vértices para dois grafos aleatórios</h3>"))
    display(df)

except ValueError as e:
    display(HTML(f"<h3>Erro: {e}</h3>"))


,Instância,Tempo Eliminação de Ciclos (s),Resultado Eliminação de Ciclos,Tempo Rótulos nos Vértices (s),Resultado Rótulos nos Vértices
0,grafo 1,0.000371,Solução ótima não encontrada.,0.000515,Solução não encontrada
1,grafo 2,0.003027,Solução ótima!,0.002199,Solução ótima!
